In [16]:
import numpy as np
import json
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import warnings
from evaluator import *

In [17]:
VARIABLES = ['ActivityCounts', 'Barometer', 'BloodPerfusion',
             'BloodPulseWave', 'EnergyExpenditure', 'GalvanicSkinResponse', 'HR',
             'HRV', 'RESP', 'Steps', 'SkinTemperature', 'ActivityClass']

# Import data

In [18]:
# file path to data folder
path = './Output'

Metadata (subjectID etc.)

In [19]:
with open(path + '/metadata_stat.txt') as f:
    metadata = f.read()

metadata = json.loads(metadata.replace('\'', '\"').replace('False', 'false').replace('True', 'true')) # doesn't accept other chars

In [20]:
subjects = [meta['subjectID'] for meta in metadata]

# XGBoost

In [21]:
class XGBoost:

    def __init__(self, path, variable):
        self.SEED = 42
        self.model = xgb.XGBClassifier(random_state=self.SEED, verbosity=0)
        self.path = path
        assert variable in (0, 1)
        self.variable = variable
        self.normalizer = StandardScaler()

    def load_data(self, indices):
        # load shape
        N = len(indices)
        N_FEATURES = np.load(self.path + '/feature_vector_stat0.npy').shape[0]

        # init
        X = np.empty((N, N_FEATURES))
        y = np.empty(N)

        # load individual datapoints
        for i, index in enumerate(indices):
            X[i, ] = np.load(path + f'/feature_vector_stat{index}.npy', allow_pickle=True)
            y[i, ] = np.load(path + f'/labels_stat{index}.npy', allow_pickle=True)[self.variable]

        return X, y

    def fit(self, train_indices):
        # load data
        X_train, y_train = self.load_data(train_indices)

        # normalize training set
        self.normalizer.fit(X_train) # fit accord. to training set
        X_train = self.normalizer.transform(X_train, copy=True)

        self.model.fit(X_train, y_train)

    def predict(self, test_indices):
        # load data
        X_test, _ = self.load_data(test_indices)

        # normalize test set
        X_test = self.normalizer.transform(X_test, copy=True)

        return self.model.predict(X_test)

# CV

In [22]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')

    for variable in (0, 1): # phF, MF
        model = XGBoost(path, variable=0)

        scores_strat_group_k_fold = stratified_group_k_fold(path=path,
                                                            groups=subjects,
                                                            model=model,
                                                            folds=5,
                                                            images=False,
                                                            verbose=True,
                                                            variable=variable)

        scores_strat_k_fold = stratified_k_fold(path=path,
                                                groups=subjects,
                                                model=model,
                                                folds=5,
                                                images=False,
                                                verbose=True,
                                                variable=variable)

        scores_loso = leave_one_subject_out(path=path,
                                                groups=subjects,
                                                model=model,
                                                images=False,
                                                verbose=True,
                                                variable=variable)

Starting stratified group 5-fold for physical fatigue


 Fold 5 F1: 0.37543391188250996: 100%|██████████| 5/5 [00:00<00:00,  6.32it/s]


Performance model:
 accuracy: 0.741 +- 0.145 

 balanced_accuracy: 0.568 +- 0.08 

 f1: 0.69 +- 0.198 

 recall: 0.741 +- 0.145 

 precision: 0.683 +- 0.219 

Starting stratified 5-fold for physical fatigue


 Fold 5 F1: 0.7210884353741497: 100%|██████████| 5/5 [00:00<00:00,  5.75it/s]


Performance model:
 accuracy: 0.789 +- 0.033 

 balanced_accuracy: 0.64 +- 0.051 

 f1: 0.765 +- 0.038 

 recall: 0.789 +- 0.033 

 precision: 0.771 +- 0.046 

Starting leave-one-subject-out for physical fatigue


...

Performance model:
 accuracy: 0.647 +- 0.137 

 balanced_accuracy: 0.504 +- 0.054 

 f1: 0.561 +- 0.167 

 recall: 0.647 +- 0.137 

 precision: 0.546 +- 0.197 

Starting stratified 5-fold for mental fatigue


 Fold 5 F1: 0.6656058323317477: 100%|██████████| 5/5 [00:00<00:00,  6.79it/s]


Performance model:
 accuracy: 0.691 +- 0.02 

 balanced_accuracy: 0.558 +- 0.025 

 f1: 0.648 +- 0.022 

 recall: 0.691 +- 0.02 

 precision: 0.652 +- 0.031 

Starting leave-one-subject-out for mental fatigue


 Fold 21 F1: 0.2727272727272727: 100%|██████████| 21/21 [00:03<00:00,  5.69it/s] 

Performance model:
 accuracy: 0.57 +- 0.263 

 balanced_accuracy: 0.496 +- 0.234 

 f1: 0.537 +- 0.273 

 recall: 0.57 +- 0.263 

 precision: 0.585 +- 0.323 

